In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
import lightgbm as lgb
from lightgbm import LGBMRegressor, Booster
import gc
from tqdm import tqdm

In [2]:
def downcast(df):
    # adapted from https://github.com/deepak7376/downcast/blob/master/src/downcast.py
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df  


In [3]:
directory = '/kaggle/input/m5-forecasting-accuracy/'
directory = 'data/'

In [4]:
calendar = pd.read_csv(f'{directory}/calendar.csv')
calendar = downcast(calendar)
calendar

Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.13 MB
Decreased by 40.4%


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [5]:
prices = pd.read_csv(f'{directory}/sell_prices.csv')
prices = downcast(prices)
prices

Memory usage of dataframe is 208.77 MB
Memory usage after optimization is: 45.76 MB
Decreased by 78.1%


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [6]:
sales = pd.read_csv(f'{directory}/sales_train_evaluation.csv')
sales = downcast(sales)
sales

Memory usage of dataframe is 452.91 MB
Memory usage after optimization is: 96.30 MB
Decreased by 78.7%


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


Create the `d` column that will be used in joining with other DataFrames

In [7]:
calendar['d'] = [f'd_{i}' for i in range(1, len(calendar) + 1)]

Although we have the `month` and `year` columns but we don't have `day`. We will make use of the full `date` to get `day`.

In [8]:
calendar['day'] = calendar['date'].dt.day.astype(np.int8)
calendar['quarter'] = calendar['date'].dt.quarter.astype(np.int8)

For the features that have cyclical characteristics, we employ a cyclical feature encoding scheme that comprises `sin` and `cos` transformations.

In [9]:
def cyclical_encode(series):
    return (series - 1) * (2 * np.pi / series.max())

In [10]:
for column_name in ['wday', 'day', 'month', 'quarter']:
    series_transformed = cyclical_encode(calendar[column_name])
    calendar[f'{column_name}_sin'] = np.sin(series_transformed).astype(np.float16)
    calendar[f'{column_name}_cos'] = np.cos(series_transformed).astype(np.float16)

Add a feature determining if the date is a weekend (either Saturday or Sunday).

In [11]:
weekend = ['Saturday', 'Sunday']
calendar['is_weekend'] = calendar['weekday'].apply(lambda x: 1 if x in weekend else 0)

In [12]:
# holiday = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', 'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', 'EidAlAdha', 'Thanksgiving', 'Christmas']
# calendar['is_holiday_1'] = calendar['event_name_1'].apply(lambda x: 1 if x in holiday else 0)
# calendar['is_holiday_2'] = calendar['event_name_2'].apply(lambda x: 1 if x in holiday else 0)
# calendar['is_holiday'] = calendar[['is_holiday_1', 'is_holiday_2']].max(axis=1)

In [13]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,quarter,wday_sin,wday_cos,day_sin,day_cos,month_sin,month_cos,quarter_sin,quarter_cos,is_weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,1,0.000000,1.000000,-0.571289,0.820801,0.0,1.000000,0.0,1.0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,1,0.781738,0.623535,-0.394287,0.918945,0.0,1.000000,0.0,1.0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,1,0.975098,-0.222534,-0.201294,0.979492,0.0,1.000000,0.0,1.0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,1,0.433838,-0.900879,0.000000,1.000000,0.5,0.866211,0.0,1.0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,1,-0.433838,-0.900879,0.201294,0.979492,0.5,0.866211,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,...,2,-0.433838,-0.900879,0.299316,-0.954102,0.5,-0.866211,1.0,0.0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,...,2,-0.975098,-0.222534,0.101196,-0.994629,0.5,-0.866211,1.0,0.0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,...,2,-0.781738,0.623535,-0.101196,-0.994629,0.5,-0.866211,1.0,0.0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,...,2,0.000000,1.000000,-0.299316,-0.954102,0.5,-0.866211,1.0,0.0,1


Drop the unnecessary `date` column as we have it separated as `day`, `month`, and `year` already.
Since the `weekday` and `wday` columns are redundant, so we drop the `weekday` column.

In [14]:
calendar.drop('date', axis=1, inplace=True, errors='ignore')
calendar.drop('weekday', axis=1, inplace=True, errors='ignore')
calendar.drop('wday', axis=1, inplace=True, errors='ignore')
calendar.drop('day', axis=1, inplace=True, errors='ignore')
calendar.drop('month', axis=1, inplace=True, errors='ignore')
calendar.drop('quarter', axis=1, inplace=True, errors='ignore')
calendar.drop('event_name_1', axis=1, inplace=True, errors='ignore')
calendar.drop('event_type_1', axis=1, inplace=True, errors='ignore')
calendar.drop('event_name_2', axis=1, inplace=True, errors='ignore')
calendar.drop('event_type_2', axis=1, inplace=True, errors='ignore')
calendar.drop('is_holiday_1', axis=1, inplace=True, errors='ignore')
calendar.drop('is_holiday_2', axis=1, inplace=True, errors='ignore')
display(calendar.head(1))
display(calendar.tail(1))

,wm_yr_wk,year,d,snap_CA,snap_TX,snap_WI,wday_sin,wday_cos,day_sin,day_cos,month_sin,month_cos,quarter_sin,quarter_cos,is_weekend
0,11101,2011,d_1,0,0,0,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1


,wm_yr_wk,year,d,snap_CA,snap_TX,snap_WI,wday_sin,wday_cos,day_sin,day_cos,month_sin,month_cos,quarter_sin,quarter_cos,is_weekend
1968,11621,2016,d_1969,0,0,0,0.781738,0.623535,-0.485352,-0.874512,0.5,-0.866211,1.0,0.0,1


Add extra days (d_1942 to d_1969) to the sales DataFrames. They serve as 28 forecast days for the evaluation rows.

In [15]:
sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1


In [16]:
zero_sales = pd.DataFrame(np.zeros((len(sales), 28), dtype=np.int16), columns=[f'd_{d}' for d in range(1942, 1969+1)])
sales = pd.concat([sales, zero_sales], axis=1)
sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Melt the DataFrame so that the time-series in wide format (along the columns) are converted to long format (each row is one time step)

In [17]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


Left join all the DataFrames to gather more features (calendar and prices)

In [18]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['item_id', 'store_id', 'wm_yr_wk'], how='left')
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,year,...,wday_sin,wday_cos,day_sin,day_cos,month_sin,month_cos,quarter_sin,quarter_cos,is_weekend,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011,...,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011,...,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011,...,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011,...,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011,...,0.0,1.0,-0.571289,0.820801,0.0,1.0,0.0,1.0,1,NaN


In [19]:
df.shape

(60034810, 23)

Drop too old sales, keep only the ones that are close to the future.

In [20]:
df.drop(df.index[df['wm_yr_wk'] <= 11430], inplace=True)
df.shape

(20123400, 23)

In [18]:
# d_id = dict(zip(df.id.cat.codes, df.id))
# d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))

# pickle.dump(d_id, file = open("d_id.pkl", "wb"))
# pickle.dump(d_store_id, file = open("d_store_id.pkl", "wb"))

To do label encoding, we encode the categorical features (those columns with the category type) using their category codes.

In [20]:
# cols = df.dtypes.index.tolist()
# types = df.dtypes.values.tolist()
# for i, (c, t) in enumerate(zip(cols, types)):
#     if t.name == 'category':
#         df[c] = df[c].cat.codes

Add combinations of mean encoding

In [22]:
# df['iteam_sold_mean'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
# df['state_sold_mean'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
# df['store_sold_mean'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
# df['cat_sold_mean'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
# df['dept_sold_mean'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
# df['cat_dept_sold_mean'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
# df['store_item_sold_mean'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
# df['cat_item_sold_mean'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
# df['dept_item_sold_mean'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
# df['state_store_sold_mean'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
# df['state_store_cat_sold_mean'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
# df['store_cat_dept_sold_mean'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

Add lag features using different shift amounts.

In [21]:
lags = [1, 7, 14, 28]
for lag in lags:
    df[f'sold_lag_{lag}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], as_index=False)['sold'].shift(lag).astype(np.float16)

Add rolling window features. We use a rolling window of 7 days (represeting one week) to generate more temporal features.

In [22]:
df['rolling_week_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['rolling_week_sold_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sold_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sold'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)

df['rolling_week_sell_price_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sell_price_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['rolling_week_sell_price_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sell_price'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sell_price_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sell_price'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)

# median?

Remove `d_` from the `d` column, so that it's no longer a string column but an integer column

In [23]:
df['d'] = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [24]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,year,...,sold_lag_14,sold_lag_28,rolling_week_sold_mean,rolling_month_sold_mean,rolling_week_sold_item_store_mean,rolling_month_sold_item_store_mean,rolling_week_sell_price_mean,rolling_month_sell_price_mean,rolling_week_sell_price_item_store_mean,rolling_month_sell_price_item_store_mean
39911410,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1310,0,11431,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39911411,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1310,1,11431,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39911412,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1310,0,11431,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39911413,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1310,3,11431,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39911414,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1310,3,11431,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# df.drop(df.index[df['d'] <= 28], inplace=True)
df.dropna(inplace=True)

In [26]:
print(df.dept_id.unique())
print(df.cat_id.unique())
print(df.store_id.unique())
print(df.state_id.unique())

['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
Categories (7, object): ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2']
['HOBBIES', 'HOUSEHOLD', 'FOODS']
Categories (3, object): ['FOODS', 'HOBBIES', 'HOUSEHOLD']
['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
Categories (10, object): ['CA_1', 'CA_2', 'CA_3', 'CA_4', ..., 'TX_3', 'WI_1', 'WI_2', 'WI_3']
['CA', 'TX', 'WI']
Categories (3, object): ['CA', 'TX', 'WI']


In [49]:
# df = pd.get_dummies(data=df, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])

In [27]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sold', 'wm_yr_wk', 'year', 'snap_CA', 'snap_TX', 'snap_WI', 'wday_sin',
       'wday_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
       'quarter_sin', 'quarter_cos', 'is_weekend', 'sell_price', 'sold_lag_1',
       'sold_lag_7', 'sold_lag_14', 'sold_lag_28', 'rolling_week_sold_mean',
       'rolling_month_sold_mean', 'rolling_week_sold_item_store_mean',
       'rolling_month_sold_item_store_mean', 'rolling_week_sell_price_mean',
       'rolling_month_sell_price_mean',
       'rolling_week_sell_price_item_store_mean',
       'rolling_month_sell_price_item_store_mean'],
      dtype='object')

In [32]:
df.to_pickle('data_train.pkl')

In [28]:
cat_features = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id']
ignore_columns = ['id', 'wm_yr_wk', 'd', 'sold']
features = df.columns[~df.columns.isin(ignore_columns)]
print(features)
X_train = df[features]
y_train = df['sold']

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'sold', 'year',
       'snap_CA', 'snap_TX', 'snap_WI', 'wday_sin', 'wday_cos', 'day_sin',
       'day_cos', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
       'is_weekend', 'sell_price', 'sold_lag_1', 'sold_lag_7', 'sold_lag_14',
       'sold_lag_28', 'rolling_week_sold_mean', 'rolling_month_sold_mean',
       'rolling_week_sold_item_store_mean',
       'rolling_month_sold_item_store_mean', 'rolling_week_sell_price_mean',
       'rolling_month_sell_price_mean',
       'rolling_week_sell_price_item_store_mean',
       'rolling_month_sell_price_item_store_mean'],
      dtype='object')


Train model

In [29]:
np.random.seed(2022)

fake_valid_index = np.random.choice(X_train.index.values, 2000000, replace=False)
train_index = np.setdiff1d(X_train.index.values, fake_valid_index)
train_data = lgb.Dataset(X_train.loc[train_index] , label=y_train.loc[train_index], categorical_feature=cat_features, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_index], label=y_train.loc[fake_valid_index], categorical_feature=cat_features, free_raw_data=False)

train_index.shape, fake_valid_index.shape

((17018076,), (2000000,))

In [30]:
odel_params = {
    'objective' : 'poisson',
    'metrics' : ['rmse', 'mae'],
    'n_estimators': 1000,
    'num_iterations' : 1000,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'num_leaves': 255,
    "min_data_in_leaf": 128,
    'min_child_weight': 300,
    "force_row_wise" : True,
    "bagging_freq" : 3,
    "bagging_fraction" : 0.5,
    "lambda_l2" : 0.1,
}

In [31]:
%%time
m_lgb = lgb.train(model_params, train_data, valid_sets=[fake_valid_data], verbose_eval=50)
m_lgb.save_model("model.lgb")

c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future 

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=0.8 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=0.8 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Info] Total Bins 5769
[LightGBM] [Info] Number of data points in the train set: 17018076, number of used features: 32


c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=0.8 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Info] Start training from score 0.193191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

Test data

In [33]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['item_id', 'store_id', 'wm_yr_wk'], how='left')
df.drop(df.index[df['wm_yr_wk'] <= 11607], inplace=True)
lags = [1, 7, 14, 28]
for lag in lags:
    df[f'sold_lag_{lag}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], as_index=False)['sold'].shift(lag).astype(np.float16)
df['rolling_week_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['rolling_week_sold_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sold_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sold'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['rolling_week_sell_price_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sell_price_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sell_price'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['rolling_week_sell_price_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sell_price'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_month_sell_price_item_store_mean'] = df.groupby(['item_id', 'store_id'])['sell_price'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)
df['d'] = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
df.dropna(inplace=True)
df.to_pickle('data_test.pkl')

In [34]:
days = np.arange(1942, 1969+1)
print(days)
for day in tqdm(days):
    X_pred = df[df['d'] == day][features]
    y_pred = m_lgb.predict(X_pred)
    df.loc[df['d'] == day, 'sold'] = y_pred

[1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955
 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969]


100%|██████████| 28/28 [00:41<00:00,  1.50s/it]


Create submission

In [37]:
sub = df[['id', 'd', 'sold']].pivot(index='id', columns='d', values='sold')
sub = sub.reset_index()
sub.columns.name = None
sub1 = pd.concat([sub.T[0:1],sub.T[-56:-28]]).T
sub2 = pd.concat([sub.T[0:1],sub.T[-28:]]).T
sub_columns = ['id'] + [f'F{i}' for i in range(1, 28+1)]
sub1.columns = sub_columns
sub2.columns = sub_columns
sub1['id'] = sub1['id'].str.replace('evaluation', 'validation')
sub = pd.concat([sub2, sub1])
sub.to_csv('submission.csv',index=False)
sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.000025,0.000024,0.000024,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000008,0.000009,0.000009,0.000009,0.000009,0.000008,0.000008,0.000008,0.000008
1,FOODS_1_001_CA_2_evaluation,0.000034,0.000029,0.000029,0.000029,0.000029,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000008,0.000008,0.000008
2,FOODS_1_001_CA_3_evaluation,0.000048,0.000043,0.000045,0.000046,0.000036,0.00004,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000008,0.000008,0.000008
3,FOODS_1_001_CA_4_evaluation,0.00003,0.000026,0.000024,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,...,0.000008,0.000009,0.000009,0.000009,0.000009,0.000009,0.000008,0.000008,0.000008,0.000008
4,FOODS_1_001_TX_1_evaluation,0.000052,0.000037,0.000038,0.000032,0.000029,0.00004,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000008,0.000008,0.000008


# Conclusions
- This M5 dataset is a huge dataset for sales prediction. It has a hierachical structure of the data, i.e., item -> dept -> store -> state.
- Working with time-series is interesting as we can harness temporal information (in this case, based on dates).
- Date-related features really need to be managed carefully. Here, I would like to highlight the use of cyclical features for dates features such as weekdays and quarters. And what about seasonality?
- Categorical features also need to be taken care of by utilizing appropriate encoding, for instance, when to use one-hot encoding (dummy encoding), or when to use label encoding.
- We can add more features like mean encoding or different statistical values (such as min, max, and median) in addition to the mentioned features.
- Although visualization can be done at different granular levels, I have not done any visualization here due to the time constraint and there are some other notebooks that did a great job in visualizing them. The point is it is a little bit difficult to plot out all the time-series (like 100k time-series in this dataset) to eyeball and find some intuitions. We can do visualizations at the highest levels of aggregration, e.g., store level, or department level.
- LightGBM is a de facto model for this kind of job. One of the pros is that it handles categorical features for us (no need to apply label encoding on them).
- We can fine tune the model by changing the hyperparameters which could give us chances to get a better performance.
- As I mentioned the hierachical structure, we may fuse the results from different granular levels using some kinds of ensemble. An example would be cranking out one model for each store and performing prediction for each store.

*Watchanan Chantapakul*

-----------------

# GARBAGE

### Load data

In [50]:
# df.to_pickle('new_data.pkl')
# del df

# import gc
# gc.collect()

In [4]:
data = pd.read_pickle('data.pkl')
# with open('d_id.pkl', 'rb') as f:
#     d_id = pickle.load(f)
# with open('d_store_id.pkl', 'rb') as f:
#     d_store_id = pickle.load(f)

In [4]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,year,...,store_sold_mean,cat_sold_mean,dept_sold_mean,cat_dept_sold_mean,store_item_sold_mean,cat_item_sold_mean,dept_item_sold_mean,state_store_sold_mean,state_store_cat_sold_mean,store_cat_dept_sold_mean
853720,14370,1437,3,1,0,0,29,0,11105,2011,...,1.307617,0.562012,0.696289,0.696289,0.326172,0.219604,0.219604,1.307617,0.801758,1.019531
853721,14380,1438,3,1,0,0,29,0,11105,2011,...,1.307617,0.562012,0.696289,0.696289,0.257568,0.263428,0.263428,1.307617,0.801758,1.019531
853722,14390,1439,3,1,0,0,29,0,11105,2011,...,1.307617,0.562012,0.696289,0.696289,0.159180,0.077820,0.077820,1.307617,0.801758,1.019531
853723,14400,1440,3,1,0,0,29,0,11105,2011,...,1.307617,0.562012,0.696289,0.696289,1.718750,2.023438,2.023438,1.307617,0.801758,1.019531
853724,14410,1441,3,1,0,0,29,0,11105,2011,...,1.307617,0.562012,0.696289,0.696289,0.972656,0.766602,0.766602,1.307617,0.801758,1.019531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,14329,1432,2,0,9,2,1969,0,11621,2016,...,1.088867,1.627930,2.039062,2.039062,0.497070,0.791016,0.791016,1.088867,1.698242,2.205078
60034806,14339,1433,2,0,9,2,1969,0,11621,2016,...,1.088867,1.627930,2.039062,2.039062,0.360107,0.425781,0.425781,1.088867,1.698242,2.205078
60034807,14349,1434,2,0,9,2,1969,0,11621,2016,...,1.088867,1.627930,2.039062,2.039062,0.826660,0.688965,0.688965,1.088867,1.698242,2.205078
60034808,14359,1435,2,0,9,2,1969,0,11621,2016,...,1.088867,1.627930,2.039062,2.039062,0.380615,0.646484,0.646484,1.088867,1.698242,2.205078


In [6]:
valid = data[(data['d'] >= 1914) & (data['d'] < 1942)][['id', 'd', 'sold']]
test = data[data['d'] >= 1942][['id', 'd', 'sold']]
eval_preds = test['sold']
valid_preds = valid['sold']

In [52]:
ignore_columns = ['id', 'wm_yr_wk', 'd']

In [56]:
# df = data
df_train = df[df['d'] < 1914]
df_valid = df[(df['d'] >= 1914) & (df['d'] < 1942)]
df_test = df[df['d'] >= 1942]
X_train, y_train = df_train.drop('sold',axis=1), df_train['sold']
X_valid, y_valid = df_valid.drop('sold', axis=1), df_valid['sold']
X_test = df_test.drop('sold', axis=1)
X_train = X_train[X_train.columns[~X_train.columns.isin(ignore_columns)]]
X_valid = X_valid[X_valid.columns[~X_valid.columns.isin(ignore_columns)]]
X_test = X_test[X_test.columns[~X_test.columns.isin(ignore_columns)]]

model = LGBMRegressor(**model_params)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse', verbose=20, early_stopping_rounds=20)
valid_preds[X_valid.index] = model.predict(X_valid)
eval_preds[X_test.index] = model.predict(X_test)

filename = f'models/allnew.txt'
model.booster_.save_model(filename, num_iteration=model.best_iteration_)
del model, X_train, y_train, X_valid, y_valid
gc.collect()

c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


MemoryError: Unable to allocate 16.8 GiB for an array with shape (45193795, 50) and data type float64

In [66]:
for store in data.store_id.unique():
    print(f'Store: {d_store_id[store]}')
    df = data[data['store_id'] == store]
    
    df_train = df[df['d'] < 1914]
    df_valid = df[(df['d'] >= 1914) & (df['d'] < 1942)]
    df_test = df[df['d'] >= 1942]
    X_train, y_train = df_train.drop('sold',axis=1), df_train['sold']
    X_valid, y_valid = df_valid.drop('sold', axis=1), df_valid['sold']
    X_test = df_test.drop('sold', axis=1)
    X_train = X_train[X_train.columns[~X_train.columns.isin(ignore_columns)]]
    X_valid = X_valid[X_valid.columns[~X_valid.columns.isin(ignore_columns)]]
    X_test = X_test[X_test.columns[~X_test.columns.isin(ignore_columns)]]
    
    model = LGBMRegressor(**model_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse', verbose=20, early_stopping_rounds=20)
    valid_preds[X_valid.index] = model.predict(X_valid)
    eval_preds[X_test.index] = model.predict(X_test)
    
    filename = f'models/{d_store_id[store]}.txt'
    model.booster_.save_model(filename, num_iteration=model.best_iteration_)
    del model, X_train, y_train, X_valid, y_valid
    gc.collect()

Store: CA_1


c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\Tommie\Miniconda3\envs\stroke\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[20]	training's rmse: 2.96919	training's l1: 1.34051	valid_1's rmse: 2.71983	valid_1's l1: 1.37399
[40]	training's rmse: 2.49121	training's l1: 1.12758	valid_1's rmse: 2.33156	valid_1's l1: 1.22764
[60]	training's rmse: 2.28175	training's l1: 1.01885	valid_1's rmse: 2.17038	valid_1's l1: 1.16169
[80]	training's rmse: 2.19527	training's l1: 0.962	valid_1's rmse: 2.10575	valid_1's l1: 1.13027
[100]	training's rmse: 2.16045	training's l1: 0.930643	valid_1's rmse: 2.07946	valid_1's l1: 1.11368
[120]	training's rmse: 2.14271	training's l1: 0.911857	valid_1's rmse: 2.06774	valid_1's l1: 1.10442
[140]	training's rmse: 2.13266	training's l1: 0.900374	valid_1's rmse: 2.06201	valid_1's l1: 1.09899
[160]	training's rmse: 2.12422	training's l1: 0.89296	valid_1's rmse: 2.05816	valid_1's l1: 1.09552
[180]	training's rmse: 2.11756	training's l1: 0.888162	valid_1's rmse: 2.0557	valid_1's l1: 1.09326
[200]	training's rmse: 2.11077	training's l1: 0.884863	valid_1's rmse: 2.05323	valid_1's l1: 1.09148
[2

In [18]:
store = 0
filename = f'models/{d_store_id[store]}.txt'
model = Booster(model_file=filename)

In [11]:
sales_valid = pd.read_csv(f'{directory}/sales_train_validation.csv')
sales_valid = downcast(sales_valid)

Memory usage of dataframe is 446.40 MB
Memory usage after optimization is: 95.16 MB
Decreased by 78.7%


In [76]:
#Set actual equal to false if you want to top in the public leaderboard :P

F_columns = [f'F{i}' for i in range(1, 28+1)]

actual = False
if not actual:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id'] + [f'd_{i}' for i in range(1914, 1941+1)]]
    validation['id'] = sales_valid.id
    validation.columns=['id'] + F_columns
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns = ['id'] + F_columns
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation') 

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns = ['id'] + F_columns
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

submit = pd.concat([validation, evaluation]).reset_index(drop=True)
submit.to_csv('submission.csv',index=False)

C:\Users\Tommie\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [22]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [23]:
submit

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,5.000000,0.000000,0.000000,...,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,0.000000,1.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,1.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,1.000000,2.000000,4.000000,1.000000,6.000000,4.000000,0.000000,...,1.000000,1.000000,0.000000,4.000000,0.000000,1.000000,3.000000,0.000000,2.000000,6.000000
4,HOBBIES_1_005_CA_1_validation,1.000000,0.000000,2.000000,3.000000,1.000000,0.000000,3.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_TX_2_evaluation,0.307572,0.217180,0.216790,0.211834,0.238596,0.263592,0.274778,0.248443,0.109544,...,0.121725,0.133629,0.133612,0.173655,0.109379,0.110822,0.110380,0.121725,0.132569,0.203148
60976,HOUSEHOLD_2_516_TX_3_evaluation,0.290923,0.250704,0.236741,0.233179,0.248818,0.293670,0.257296,0.102082,0.102169,...,0.108045,0.168522,0.118063,0.102692,0.100708,0.101960,0.100746,0.154889,0.169517,0.186471
60977,HOUSEHOLD_2_516_WI_1_evaluation,0.155159,0.131516,0.119009,0.168832,0.168165,0.118864,0.115088,0.091193,0.088102,...,0.112460,0.121205,0.116681,0.089172,0.088156,0.090149,0.089215,0.170160,0.120449,0.104158
60978,HOUSEHOLD_2_516_WI_2_evaluation,0.067310,0.049355,0.049659,0.050881,0.053906,0.057554,0.067968,0.062183,0.050812,...,0.053897,0.056990,0.067451,0.063819,0.049007,0.050093,0.050864,0.053609,0.056902,0.051887


In [71]:
actual_valid = sales[[f'd_{i}' for i in range(1914, 1941+1)]].to_numpy()

In [39]:
valid['sold'] = valid_preds
validation = valid[['id','d','sold']]
validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
validation.columns = ['id'] + F_columns
validation.id = validation.id.map(d_id).str.replace('evaluation','validation') 

In [77]:
predicted_valid = validation.drop('id', axis=1).to_numpy()